## Task 2: Claim Normalization

First, we install all necessary libraries and packages

In [ ]:
!pip install datasets
!pip install evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 13.5 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; platform_system ==

In [ ]:
pip install rouge_score sacrebleu bert_score

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.1/104.1 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 52.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 42.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 47.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 10.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.0 MB/s eta 0:00:00
   

In [ ]:
# Import libraries
import re
import pandas as pd
from sklearn.model_selection import train_test_split
import torch
from transformers import (AutoTokenizer, AutoModelForSeq2SeqLM,
                          Seq2SeqTrainingArguments, Seq2SeqTrainer,
                          DataCollatorForSeq2Seq)
from datasets import Dataset
import evaluate

In [ ]:
# 1. Preprocessing Functions

def preprocess_text(text):
    """
    Expand common contractions and abbreviations,
    remove URLs, special characters, extra whitespace,
    and convert text to lowercase.
    """
    # Abbreviation mappings
    contraction_map = {
        "he'll": "he will",
        "she's": "she is",
        "gov.": "governor",
        "feb.": "february",
        "vp": "vice president",
        "eta": "estimated time of arrival"
    }

    # Replace each abbreviation by referring to the mapping
    for contraction, expansion in contraction_map.items():
        text = re.sub(r'\b' + re.escape(contraction) + r'\b', expansion, text, flags=re.IGNORECASE)

    # Remove URLs
    text = re.sub(r'http\S+|www.\S+', '', text)
    # Remove special characters
    text = re.sub(r'[^a-zA-Z0-9\s]', '', text)
    # Remove extra whitespace
    text = re.sub(r'\s+', ' ', text)
    # Convert to lowercase and strip leading/trailing whitespace
    return text.lower().strip()


# 2. Data Loading and Splitting

def load_and_split_data(filepath):
    """
    Load the dataset from CSV, preprocess the text fields,
    and split the data into training (70%), validation (15%), and test (15%).
    Assumes the CSV has columns "text" (social media post) and
    "normalized_claim" (target normalized claim).
    """
    df = pd.read_csv(filepath)

    # Preprocess both the social media post and normalized claim
    df['Social Media Post'] = df['Social Media Post'].apply(preprocess_text)
    df['Normalized Claim'] = df['Normalized Claim'].apply(preprocess_text)

    df.rename(columns={"Social Media Post": "text", "Normalized Claim": "normalized_claim"}, inplace=True)

    # Data splitting
    train_df, temp_df = train_test_split(df, test_size=0.30, random_state=42)
    val_df, test_df = train_test_split(temp_df, test_size=0.5, random_state=42)

    return train_df, val_df, test_df

def convert_to_dataset(df):
    """Convert a Pandas DataFrame into a Hugging Face Dataset."""
    return Dataset.from_pandas(df)


# 3. Tokenization

def tokenize_function(example, tokenizer, max_input_length=128, max_target_length=128):
    """
    Tokenize the input text and target normalized claim.
    """
    model_inputs = tokenizer(example['text'], max_length=max_input_length, truncation=True)
    # Tokenize target text using the tokenizer’s target mode
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(example['normalized_claim'], max_length=max_target_length, truncation=True)
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs


# 4. Evaluation Metrics Function

def safe_decode(token_ids, tokenizer, ignore_idx=-100):
    """Remove None and negative IDs,
    then decode the remainder.
    """
    safe_tokens = []
    vocab_size = tokenizer.vocab_size
    for t in token_ids:
        if t is None:
            continue
        # Convert to Python int
        t = int(t)
        # If it is the ignore index, convert to pad token
        if t == ignore_idx:
            t = tokenizer.pad_token_id
        # If it is out of range, skip it
        if t < 0:
            t = 0
        if t >= vocab_size:
            t = vocab_size - 1
        safe_tokens.append(t)
    return tokenizer.decode(safe_tokens, skip_special_tokens=True)


def compute_metrics(eval_pred, tokenizer):
    """
    Compute the ROUGE-L, BLEU, and BERTScore metrics.
    """
    predictions, labels = eval_pred

    # Convert predictions and labels from tensor
    if isinstance(predictions, torch.Tensor):
        predictions = predictions.tolist()
    if isinstance(labels, torch.Tensor):
        labels = labels.tolist()

    decoded_preds = [safe_decode(seq, tokenizer) for seq in predictions]
    decoded_labels = [safe_decode(seq, tokenizer) for seq in labels]

    # Evaluate
    rouge = evaluate.load("rouge")
    bleu = evaluate.load("bleu")
    bertscore = evaluate.load("bertscore")

    rouge_output = rouge.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)
    bleu_output = bleu.compute(predictions=decoded_preds, references=decoded_labels)
    bertscore_output = bertscore.compute(predictions=decoded_preds, references=decoded_labels, lang="en")

    avg_bertscore = sum(bertscore_output["f1"]) / len(bertscore_output["f1"])

    return {
        "rougeL": rouge_output["rougeL"],
        "bleu": bleu_output["bleu"],
        "bertscore": avg_bertscore
    }

def compute_metrics_wrapper(eval_pred):
    return compute_metrics(eval_pred, compute_metrics_wrapper.tokenizer)

# 5. Model Training Function

def train_model(model_checkpoint, train_dataset, val_dataset, tokenizer):
    """
    Fine-tune a sequence-to-sequence model using Hugging Face Trainer.
    """
    model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)

    # Tokenize datasets
    tokenized_train = train_dataset.map(lambda x: tokenize_function(x, tokenizer), batched=True)
    tokenized_val = val_dataset.map(lambda x: tokenize_function(x, tokenizer), batched=True)

    # Define training arguments
    training_args = Seq2SeqTrainingArguments(
        output_dir=f'./results_{model_checkpoint.replace("/", "_")}',
        evaluation_strategy="epoch",
        learning_rate=2e-5,
        per_device_train_batch_size=4,
        per_device_eval_batch_size=4,
        weight_decay=0.01,
        save_total_limit=3,
        num_train_epochs=3,
        predict_with_generate=True,
        fp16=torch.cuda.is_available()
    )

    data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

    # Set the tokenizer for the metric function wrapper
    compute_metrics_wrapper.tokenizer = tokenizer

    trainer = Seq2SeqTrainer(
        model=model,
        args=training_args,
        train_dataset=tokenized_train,
        eval_dataset=tokenized_val,
        data_collator=data_collator,
        tokenizer=tokenizer,
        compute_metrics=compute_metrics_wrapper
    )

    trainer.train()

    # Evaluate on validation set
    metrics = trainer.evaluate()
    print(f"Validation metrics for {model_checkpoint}:\n{metrics}")

    # Save the trained model
    save_dir = f"./best_model_{model_checkpoint.replace('/', '_')}"
    trainer.save_model(save_dir)
    return model, metrics, save_dir


# 6. Inference Function

def inference(model, tokenizer, test_dataset, max_input_length=128, max_target_length=128):
    predictions = []
    for example in test_dataset:
        inputs = tokenizer(example["text"], return_tensors="pt", max_length=max_input_length, truncation=True)
        if torch.cuda.is_available():
            inputs = {k: v.cuda() for k, v in inputs.items()}
            model.cuda()

        output_ids = model.generate(**inputs, max_length=max_target_length)[0].cpu().tolist()
        pred = safe_decode(output_ids, tokenizer)
        predictions.append(pred)
    return predictions


In [ ]:
# 7. Main Pipeline

def main():
    # Load and split the data
    train_df, val_df, test_df = load_and_split_data("/content/drive/MyDrive/CLAN_data.csv")
    train_dataset = convert_to_dataset(train_df)
    val_dataset = convert_to_dataset(val_df)
    test_dataset = convert_to_dataset(test_df)

    # Define the models to fine-tune
    model_checkpoints = ["facebook/bart-large", "t5-base"]
    results = {}
    model_save_dirs = {}

    for checkpoint in model_checkpoints:
        print(f"\nTraining model: {checkpoint}")
        tokenizer = AutoTokenizer.from_pretrained(checkpoint, use_fast=False)
        model, metrics, save_dir = train_model(checkpoint, train_dataset, val_dataset, tokenizer)
        results[checkpoint] = metrics
        model_save_dirs[checkpoint] = save_dir

    # Best model based on the ROUGE-L score
    best_model_checkpoint = max(results, key=lambda k: results[k].get("eval_rougeL", 0))
    print(f"\nBest model based on ROUGE-L: {best_model_checkpoint}")

    # Load the best model and its tokenizer for inference
    best_model = AutoModelForSeq2SeqLM.from_pretrained(model_save_dirs[best_model_checkpoint])
    best_tokenizer = AutoTokenizer.from_pretrained(best_model_checkpoint)

    # Run inference on the test set
    preds = inference(best_model, best_tokenizer, test_dataset)
    # Save predictions to a CSV file
    test_df["generated_normalized_claim"] = preds
    test_df.to_csv("test_predictions.csv", index=False)
    print("\nTest predictions saved to 'test_predictions.csv'.")

if __name__ == "__main__":
    main()


Training model: facebook/bart-large


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.63k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

...

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1611: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
<ipython-input-4-b125614ad09e>:178: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(
Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.48.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.


Epoch,Training Loss,Validation Loss,Rougel,Bleu,Bertscore
1,No log,2.507727,0.286100,0.148031,0.860970
2,2.786300,2.420529,0.293681,0.152825,0.865056
3,2.471900,2.408035,0.293599,0.152949,0.865037


Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Validation metrics for t5-base:
{'eval_loss': 2.4080348014831543, 'eval_rougeL': 0.29359866307874966, 'eval_bleu': 0.15294861090464817, 'eval_bertscore': 0.865036691675819, 'eval_runtime': 73.0971, 'eval_samples_per_second': 5.773, 'eval_steps_per_second': 1.45, 'epoch': 3.0}

Best model based on ROUGE-L: facebook/bart-large

Test predictions saved to 'test_predictions.csv'.


In [ ]:
def evaluate_trained_model(
    model_folder_path,
    test_csv_path,
    output_csv="test_predictions.csv",
    max_input_length=128,
    max_target_length=128
):
    """
    1. Loads the trained model and tokenizer from disk.
    2. Reads and preprocesses the test CSV.
    3. Generates normalized claims.
    4. Computes ROUGE-L, BLEU, and BERTScore.
    """
    model = AutoModelForSeq2SeqLM.from_pretrained(model_folder_path)
    tokenizer = AutoTokenizer.from_pretrained(model_folder_path, use_fast=False)

    df = pd.read_csv(test_csv_path)

    df["Social Media Post"] = df["Social Media Post"].apply(preprocess_text)
    df["Normalized Claim"] = df["Normalized Claim"].apply(preprocess_text)

    df.rename(
        columns={"Social Media Post": "text", "Normalized Claim": "normalized_claim"},
        inplace=True
    )

    test_dataset = Dataset.from_pandas(df)

    preds = inference(model, tokenizer, test_dataset, max_input_length, max_target_length)
    df["generated_normalized_claim"] = preds

    references = df["normalized_claim"].tolist()
    predictions = df["generated_normalized_claim"].tolist()

    rouge = evaluate.load("rouge")
    bleu = evaluate.load("bleu")
    bertscore = evaluate.load("bertscore")

    rouge_output = rouge.compute(predictions=predictions, references=references, use_stemmer=True)
    bleu_output = bleu.compute(predictions=predictions, references=references)
    bertscore_output = bertscore.compute(predictions=predictions, references=references, lang="en")

    avg_bertscore = sum(bertscore_output["f1"]) / len(bertscore_output["f1"])

    metrics = {
        "rougeL": rouge_output["rougeL"],
        "bleu": bleu_output["bleu"],
        "bertscore": avg_bertscore
    }

    df.to_csv(output_csv, index=False)

    print("Evaluation on test.csv complete:")
    print(f"ROUGE-L: {metrics['rougeL']:.4f}")
    print(f"BLEU: {metrics['bleu']:.4f}")
    print(f"BERTScore: {metrics['bertscore']:.4f}")
    print(f"Predictions saved to: {output_csv}")

    return metrics
